In [9]:
# %pip install geocoder rouge
import numpy as np
import pandas as pd
import nltk
import random
import time
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import math
from rouge import Rouge

In [10]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

### 1.Read in Dataset and Construct the Vocabulary

In [11]:
word2index = {}
index2word = {}


def read_data(file='Data.csv'):
    data = pd.read_csv(file)
    summarys = []
    contents = []
    for i, row in data.iterrows():
        content = []
        summary = []
        for word in nltk.word_tokenize(str(row['Content'])):
            if word not in word2index:
                word2index[word] = len(word2index)
            content.append(word2index[word])
        for word in nltk.word_tokenize(
                str(row['Title']) + str(row['Subtitle'])):
            if word not in word2index:
                word2index[word] = len(word2index)
            summary.append(word2index[word])
        summarys.append(summary)
        contents.append(content)
    return contents, summarys

In [12]:
contents, summarys = read_data('Data.csv')

In [13]:
word2index['PAD'] = len(word2index)
word2index['BOS'] = len(word2index)
word2index['EOS'] = len(word2index)
index2word = {v: k for k, v in word2index.items()}

In [14]:
word2index

{'SINGAPORE': 0,
 ':': 1,
 'Singapore': 2,
 "'s": 3,
 'funeral': 4,
 'industry': 5,
 'is': 6,
 'looking': 7,
 'to': 8,
 'fill': 9,
 'at': 10,
 'least': 11,
 '800': 12,
 'vacancies': 13,
 'over': 14,
 'the': 15,
 'next': 16,
 'few': 17,
 'years': 18,
 ',': 19,
 'amid': 20,
 'an': 21,
 'expected': 22,
 'rise': 23,
 'in': 24,
 'demand': 25,
 'for': 26,
 'such': 27,
 'services': 28,
 '.': 29,
 'The': 30,
 'country': 31,
 'annual': 32,
 'deaths': 33,
 'have': 34,
 'been': 35,
 'climbing': 36,
 'steadily': 37,
 'with': 38,
 'ageing': 39,
 'population': 40,
 'They': 41,
 'are': 42,
 'set': 43,
 'nearly': 44,
 'double': 45,
 'from': 46,
 'about': 47,
 '24,000': 48,
 '2021': 49,
 '40,000': 50,
 '2040': 51,
 'according': 52,
 'National': 53,
 'Environment': 54,
 'Agency': 55,
 '(': 56,
 'NEA': 57,
 ')': 58,
 'Funeral': 59,
 'firms': 60,
 'hire': 61,
 'a': 62,
 'new': 63,
 'generation': 64,
 'of': 65,
 'workers': 66,
 'In-demand': 67,
 'positions': 68,
 'include': 69,
 'directors': 70,
 'and': 71

### 2.Define RNN Encoder

In [15]:
class AINLPEncoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.RNN(emb_dim, hid_dim, n_layers, dropout=dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):

        embedded = self.dropout(self.embedding(src))
        outputs, hidden = self.rnn(embedded)

        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        #outputs are always from the top hidden layer

        return outputs, hidden

### 3.Define RNN Decoder

In [16]:
class AINLPDecoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()

        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers

        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.RNN(emb_dim, hid_dim, n_layers, dropout=dropout)
        self.fc_out = nn.Linear(hid_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden):

        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]

        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]

        input = input.unsqueeze(0)

        #input = [1, batch size]

        embedded = self.dropout(self.embedding(input))

        #embedded = [1, batch size, emb dim]

        output, hidden = self.rnn(embedded, hidden)

        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]

        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]

        prediction = self.fc_out(output.squeeze(0))

        #prediction = [batch size, output dim]

        return prediction, hidden

### 4.Define Seq2Seq Model

In [17]:
class AINLPSeq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"

    def forward(self, src, trg, teacher_forcing_ratio=0.5):

        #src = [src len, batch size]
        #trg = [trg len, batch size]

        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim

        outputs = torch.zeros(trg_len, batch_size,
                              trg_vocab_size).to(self.device)

        output, hidden = self.encoder(src)

        input = trg[0, :]

        for t in range(1, trg_len):

            # Get the output from decoder
            output, hidden = self.decoder(input, hidden)

            # Repalce the output at time t
            outputs[t] = output

            # Teacher Forcing
            teacher_force = random.random() < teacher_forcing_ratio

            # Get the output with maximal probability
            top1 = output.argmax(1)

            # The new one as the input of next step
            input = trg[t] if teacher_force else top1

        return outputs

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### 5.Initialize the Model with Random Weights

In [19]:
INPUT_DIM = len(word2index)
OUTPUT_DIM = len(word2index)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = AINLPEncoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = AINLPDecoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = AINLPSeq2Seq(enc, dec, device).to(device)

In [20]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

AINLPSeq2Seq(
  (encoder): AINLPEncoder(
    (embedding): Embedding(30372, 256)
    (rnn): RNN(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): AINLPDecoder(
    (embedding): Embedding(30372, 256)
    (rnn): RNN(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=30372, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [21]:
optimizer = optim.Adam(model.parameters())

### 6.Data Padding to Form Batch

In [22]:
max_src = min(max(map(len, contents)), 512)
max_tgt = min(max(map(len, summarys)), 128)

In [23]:
max_src, max_tgt

(512, 75)

In [24]:
sources = []
targets = []
for i in contents:
    if len(i) < max_src:
        i += [word2index['PAD']] * (max_src - len(i))
    sources.append(i[:max_src])

max_tgt += 2
for i in summarys:
    if len(i) < max_tgt:
        i = [word2index['BOS']] + i + [word2index['EOS']]
        i += [word2index['PAD']] * (max_src - len(i))
    targets.append(i[:max_tgt])

In [25]:
class CuDataset(Dataset):
    def __init__(self, sources, targets):
        self.sources = sources
        self.targets = targets

    def __getitem__(self, idx):
        idx = int(idx)
        item = {}
        item['src'] = torch.tensor(self.sources[idx])
        item['tgt'] = torch.tensor(self.targets[idx])
        return item

    def __len__(self):
        return len(self.sources)

In [26]:
len(sources), len(targets)

(806, 806)

In [27]:
train_sources, test_sources, train_targets, test_targets = train_test_split(
    sources, targets, test_size=0.1, random_state=43)

In [28]:
train_dataset = CuDataset(train_sources, train_targets)
val_dataset = CuDataset(test_sources, test_targets)
batch_size = 2
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
eval_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [29]:
len(val_dataset)

81

### 7.Train

In [30]:
criterion = nn.CrossEntropyLoss(ignore_index=word2index['PAD'])

In [31]:
@torch.no_grad()
def evaluate(model, dataloader, criterion):

    model.eval()

    epoch_loss = 0

    with torch.no_grad():

        for i, batch in enumerate(dataloader):

            src = batch['src'].transpose(0, 1)
            trg = batch['tgt'].transpose(0, 1)

            input_tgt = trg[:-1]
            label_tgt = trg[1:]
            output = model(src, input_tgt, 0)  #turn off teacher forcing

            output_dim = output.shape[-1]

            output = output.view(-1, output_dim)
            label_tgt = label_tgt.contiguous().view(-1)

            loss = criterion(output, label_tgt)

            print(
                f'[Step] Train Loss: {loss:.3f} | Train PPL: {math.exp(loss):7.3f}'
            )

            epoch_loss += loss.item()

    return epoch_loss / len(dataloader)

In [32]:
def train(model, iterator, optimizer, criterion, clip):

    model.train()

    epoch_loss = 0

    for i, batch in enumerate(iterator):

        src = batch['src'].transpose(0, 1)
        trg = batch['tgt'].transpose(0, 1)

        optimizer.zero_grad()

        input_tgt = trg[:-1]
        label_tgt = trg[1:]

        output = model(src, input_tgt)
        output_dim = output.shape[-1]

        output = output.view(-1, output_dim)
        trg = label_tgt.contiguous().view(-1)

        loss = criterion(output, trg)
        
        print(
        f'[Step] Train Loss: {loss:.3f} | Train PPL: {math.exp(loss):7.3f}')

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [33]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [34]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss = train(model, train_loader, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, eval_loader, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'model.pt')

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(
        f'[Epoch] Train Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}'
    )
    print(
        f'[Epoch] Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}'
    )

[Step] Train Loss: 10.440 | Train PPL: 34205.563
[Step] Train Loss: 10.474 | Train PPL: 35373.548
[Step] Train Loss: 10.480 | Train PPL: 35604.574
[Step] Train Loss: 10.310 | Train PPL: 30033.282
[Step] Train Loss: 10.434 | Train PPL: 33998.592
[Step] Train Loss: 10.201 | Train PPL: 26929.954
[Step] Train Loss: 9.971 | Train PPL: 21395.172
[Step] Train Loss: 9.644 | Train PPL: 15428.790
[Step] Train Loss: 9.468 | Train PPL: 12938.656
[Step] Train Loss: 9.288 | Train PPL: 10809.012
[Step] Train Loss: 9.623 | Train PPL: 15108.367
[Step] Train Loss: 9.290 | Train PPL: 10826.571
[Step] Train Loss: 8.979 | Train PPL: 7938.229
[Step] Train Loss: 8.103 | Train PPL: 3305.954
[Step] Train Loss: 8.100 | Train PPL: 3294.432
[Step] Train Loss: 8.920 | Train PPL: 7483.715
[Step] Train Loss: 9.214 | Train PPL: 10037.197
[Step] Train Loss: 8.427 | Train PPL: 4568.094
[Step] Train Loss: 8.774 | Train PPL: 6467.196
[Step] Train Loss: 8.854 | Train PPL: 7002.243
[Step] Train Loss: 9.466 | Train PPL: 129

[Step] Train Loss: 8.663 | Train PPL: 5784.378
[Step] Train Loss: 8.386 | Train PPL: 4385.458
[Step] Train Loss: 8.247 | Train PPL: 3815.617
[Step] Train Loss: 8.623 | Train PPL: 5559.955
[Step] Train Loss: 8.087 | Train PPL: 3251.430
[Step] Train Loss: 8.662 | Train PPL: 5778.402
[Step] Train Loss: 8.011 | Train PPL: 3013.552
[Step] Train Loss: 9.804 | Train PPL: 18098.168
[Step] Train Loss: 8.598 | Train PPL: 5421.110
[Step] Train Loss: 7.911 | Train PPL: 2727.354
[Step] Train Loss: 9.188 | Train PPL: 9781.206
[Step] Train Loss: 8.737 | Train PPL: 6227.080
[Step] Train Loss: 8.837 | Train PPL: 6881.052
[Step] Train Loss: 8.396 | Train PPL: 4429.081
[Step] Train Loss: 8.530 | Train PPL: 5063.208
[Step] Train Loss: 7.872 | Train PPL: 2621.661
[Step] Train Loss: 8.778 | Train PPL: 6492.595
[Step] Train Loss: 7.771 | Train PPL: 2371.281
[Step] Train Loss: 8.159 | Train PPL: 3493.788
[Step] Train Loss: 8.700 | Train PPL: 6001.760
[Step] Train Loss: 8.737 | Train PPL: 6228.773
[Step] Train

[Step] Train Loss: 8.089 | Train PPL: 3258.057
[Step] Train Loss: 7.441 | Train PPL: 1703.647
[Step] Train Loss: 8.109 | Train PPL: 3325.442
[Step] Train Loss: 8.540 | Train PPL: 5115.081
[Step] Train Loss: 7.185 | Train PPL: 1320.005
[Step] Train Loss: 8.467 | Train PPL: 4757.487
[Step] Train Loss: 9.287 | Train PPL: 10793.036
[Step] Train Loss: 8.447 | Train PPL: 4661.498
[Step] Train Loss: 8.694 | Train PPL: 5967.004
[Step] Train Loss: 10.820 | Train PPL: 50002.488
[Step] Train Loss: 8.876 | Train PPL: 7155.640
[Step] Train Loss: 8.165 | Train PPL: 3514.407
[Step] Train Loss: 8.015 | Train PPL: 3024.579
[Step] Train Loss: 8.513 | Train PPL: 4979.000
[Step] Train Loss: 8.842 | Train PPL: 6921.258
[Step] Train Loss: 7.940 | Train PPL: 2806.804
[Step] Train Loss: 9.361 | Train PPL: 11620.689
[Step] Train Loss: 8.354 | Train PPL: 4248.817
[Step] Train Loss: 8.416 | Train PPL: 4519.022
[Step] Train Loss: 8.153 | Train PPL: 3473.874
[Step] Train Loss: 8.662 | Train PPL: 5778.617
[Step] Tr

[Step] Train Loss: 8.046 | Train PPL: 3120.027
[Step] Train Loss: 8.122 | Train PPL: 3366.390
[Step] Train Loss: 8.222 | Train PPL: 3722.845
[Step] Train Loss: 8.082 | Train PPL: 3234.880
[Step] Train Loss: 7.951 | Train PPL: 2838.085
[Step] Train Loss: 7.582 | Train PPL: 1962.611
[Step] Train Loss: 7.894 | Train PPL: 2681.592
[Step] Train Loss: 7.644 | Train PPL: 2087.842
[Step] Train Loss: 8.235 | Train PPL: 3770.315
[Step] Train Loss: 7.615 | Train PPL: 2027.463
[Step] Train Loss: 8.173 | Train PPL: 3544.401
[Step] Train Loss: 7.458 | Train PPL: 1733.010
[Step] Train Loss: 8.244 | Train PPL: 3806.200
[Step] Train Loss: 7.638 | Train PPL: 2075.483
[Step] Train Loss: 8.527 | Train PPL: 5050.371
[Step] Train Loss: 7.911 | Train PPL: 2728.277
[Step] Train Loss: 7.877 | Train PPL: 2636.477
[Step] Train Loss: 8.586 | Train PPL: 5358.703
[Step] Train Loss: 7.351 | Train PPL: 1557.786
[Step] Train Loss: 7.982 | Train PPL: 2927.702
[Step] Train Loss: 7.761 | Train PPL: 2348.318
[Step] Train 

[Step] Train Loss: 8.034 | Train PPL: 3084.069
[Step] Train Loss: 7.877 | Train PPL: 2636.184
[Step] Train Loss: 7.530 | Train PPL: 1862.926
[Step] Train Loss: 8.265 | Train PPL: 3887.216
[Step] Train Loss: 7.775 | Train PPL: 2379.283
[Step] Train Loss: 7.469 | Train PPL: 1752.671
[Step] Train Loss: 7.508 | Train PPL: 1822.235
[Step] Train Loss: 7.589 | Train PPL: 1976.390
[Step] Train Loss: 7.826 | Train PPL: 2503.697
[Step] Train Loss: 8.906 | Train PPL: 7374.432
[Step] Train Loss: 7.821 | Train PPL: 2492.845
[Step] Train Loss: 7.237 | Train PPL: 1389.250
[Step] Train Loss: 7.861 | Train PPL: 2592.940
[Step] Train Loss: 8.585 | Train PPL: 5348.222
[Step] Train Loss: 7.642 | Train PPL: 2083.934
[Step] Train Loss: 8.648 | Train PPL: 5696.981
[Step] Train Loss: 7.482 | Train PPL: 1775.366
[Step] Train Loss: 7.964 | Train PPL: 2874.214
[Step] Train Loss: 7.447 | Train PPL: 1714.930
[Step] Train Loss: 7.773 | Train PPL: 2376.359
[Step] Train Loss: 8.104 | Train PPL: 3306.030
[Step] Train 

...

[Step] Train Loss: 7.903 | Train PPL: 2705.942
[Step] Train Loss: 7.923 | Train PPL: 2760.800
[Step] Train Loss: 8.107 | Train PPL: 3317.355
[Step] Train Loss: 8.074 | Train PPL: 3209.409
[Step] Train Loss: 7.563 | Train PPL: 1925.847
[Step] Train Loss: 7.622 | Train PPL: 2043.295
[Step] Train Loss: 7.586 | Train PPL: 1970.792
[Step] Train Loss: 7.785 | Train PPL: 2405.406
[Step] Train Loss: 7.742 | Train PPL: 2303.479
[Step] Train Loss: 8.101 | Train PPL: 3297.172
[Step] Train Loss: 7.923 | Train PPL: 2760.137
[Step] Train Loss: 7.906 | Train PPL: 2712.435
[Step] Train Loss: 8.476 | Train PPL: 4798.135
[Step] Train Loss: 7.810 | Train PPL: 2465.938
[Step] Train Loss: 7.901 | Train PPL: 2698.891
[Step] Train Loss: 7.910 | Train PPL: 2724.380
[Step] Train Loss: 7.358 | Train PPL: 1569.266
[Step] Train Loss: 7.393 | Train PPL: 1624.260
[Step] Train Loss: 7.946 | Train PPL: 2824.774
[Step] Train Loss: 8.064 | Train PPL: 3177.901
[Step] Train Loss: 7.659 | Train PPL: 2118.944
[Step] Train 

[Step] Train Loss: 8.323 | Train PPL: 4117.502
[Step] Train Loss: 7.829 | Train PPL: 2511.975
[Step] Train Loss: 8.344 | Train PPL: 4203.456
[Step] Train Loss: 8.108 | Train PPL: 3321.347
[Step] Train Loss: 7.914 | Train PPL: 2734.557
[Step] Train Loss: 8.137 | Train PPL: 3418.181
[Step] Train Loss: 7.458 | Train PPL: 1733.968
[Step] Train Loss: 7.803 | Train PPL: 2447.108
[Step] Train Loss: 7.823 | Train PPL: 2497.963
[Step] Train Loss: 7.876 | Train PPL: 2634.051
[Step] Train Loss: 8.031 | Train PPL: 3075.968
[Step] Train Loss: 7.854 | Train PPL: 2576.233
[Step] Train Loss: 7.173 | Train PPL: 1303.402
[Step] Train Loss: 7.539 | Train PPL: 1879.243
[Step] Train Loss: 7.978 | Train PPL: 2917.029
[Step] Train Loss: 7.121 | Train PPL: 1237.897
[Step] Train Loss: 8.240 | Train PPL: 3790.251
[Step] Train Loss: 8.361 | Train PPL: 4277.618
[Step] Train Loss: 7.716 | Train PPL: 2244.530
[Step] Train Loss: 8.239 | Train PPL: 3786.801
[Step] Train Loss: 7.280 | Train PPL: 1451.194
[Step] Train 

[Step] Train Loss: 7.959 | Train PPL: 2860.398
[Step] Train Loss: 8.145 | Train PPL: 3444.839
[Step] Train Loss: 7.548 | Train PPL: 1897.176
[Step] Train Loss: 7.266 | Train PPL: 1430.556
[Step] Train Loss: 7.958 | Train PPL: 2859.293
[Step] Train Loss: 7.276 | Train PPL: 1445.148
[Step] Train Loss: 7.893 | Train PPL: 2679.060
[Step] Train Loss: 8.241 | Train PPL: 3794.110
[Step] Train Loss: 7.703 | Train PPL: 2214.589
[Step] Train Loss: 7.641 | Train PPL: 2081.811
[Step] Train Loss: 8.020 | Train PPL: 3042.661
[Step] Train Loss: 7.501 | Train PPL: 1809.554
[Step] Train Loss: 7.273 | Train PPL: 1441.133
[Step] Train Loss: 8.083 | Train PPL: 3238.831
[Step] Train Loss: 7.898 | Train PPL: 2691.267
[Step] Train Loss: 7.538 | Train PPL: 1877.737
[Step] Train Loss: 7.745 | Train PPL: 2309.341
[Step] Train Loss: 8.400 | Train PPL: 4447.985
[Step] Train Loss: 7.908 | Train PPL: 2720.272
[Step] Train Loss: 8.368 | Train PPL: 4306.159
[Step] Train Loss: 7.707 | Train PPL: 2223.017
[Step] Train 

[Step] Train Loss: 7.218 | Train PPL: 1363.627
[Step] Train Loss: 8.124 | Train PPL: 3375.350
[Step] Train Loss: 7.227 | Train PPL: 1376.130
[Step] Train Loss: 7.668 | Train PPL: 2138.023
[Step] Train Loss: 8.209 | Train PPL: 3672.695
[Step] Train Loss: 7.548 | Train PPL: 1897.705
[Step] Train Loss: 7.796 | Train PPL: 2431.390
[Step] Train Loss: 7.988 | Train PPL: 2945.865
[Step] Train Loss: 7.613 | Train PPL: 2024.363
[Step] Train Loss: 7.893 | Train PPL: 2679.366
[Step] Train Loss: 7.895 | Train PPL: 2683.814
[Step] Train Loss: 8.175 | Train PPL: 3552.285
[Step] Train Loss: 7.685 | Train PPL: 2174.825
[Step] Train Loss: 8.534 | Train PPL: 5085.411
[Step] Train Loss: 7.883 | Train PPL: 2652.864
[Step] Train Loss: 8.505 | Train PPL: 4940.969
[Step] Train Loss: 7.538 | Train PPL: 1877.836
[Step] Train Loss: 8.065 | Train PPL: 3180.876
[Step] Train Loss: 8.137 | Train PPL: 3419.185
[Step] Train Loss: 7.549 | Train PPL: 1897.907
[Step] Train Loss: 7.574 | Train PPL: 1947.845
[Step] Train 

[Step] Train Loss: 8.338 | Train PPL: 4180.536
[Step] Train Loss: 7.753 | Train PPL: 2327.929
[Step] Train Loss: 7.524 | Train PPL: 1852.847
[Step] Train Loss: 8.363 | Train PPL: 4284.273
[Step] Train Loss: 10.764 | Train PPL: 47292.263
[Step] Train Loss: 8.592 | Train PPL: 5386.432
[Step] Train Loss: 8.827 | Train PPL: 6812.914
[Step] Train Loss: 9.119 | Train PPL: 9126.282
[Step] Train Loss: 8.869 | Train PPL: 7110.742
[Step] Train Loss: 8.488 | Train PPL: 4854.000
[Step] Train Loss: 8.828 | Train PPL: 6819.662
[Step] Train Loss: 8.182 | Train PPL: 3574.793
[Step] Train Loss: 8.372 | Train PPL: 4325.504
[Step] Train Loss: 10.242 | Train PPL: 28046.928
[Step] Train Loss: 9.456 | Train PPL: 12790.074
[Step] Train Loss: 9.162 | Train PPL: 9530.154
[Step] Train Loss: 8.686 | Train PPL: 5920.947
[Step] Train Loss: 9.098 | Train PPL: 8936.510
[Step] Train Loss: 8.235 | Train PPL: 3771.861
[Step] Train Loss: 9.781 | Train PPL: 17698.746
[Step] Train Loss: 9.735 | Train PPL: 16907.148
[Step]

### 8.Abstract Generation and Performance Evaluation

In [35]:
model.load_state_dict(torch.load('model.pt'))

<All keys matched successfully>

In [36]:
model.eval()

AINLPSeq2Seq(
  (encoder): AINLPEncoder(
    (embedding): Embedding(30372, 256)
    (rnn): RNN(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): AINLPDecoder(
    (embedding): Embedding(30372, 256)
    (rnn): RNN(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=30372, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [37]:
@torch.no_grad()
def generate(model, dataloader, max_len=50):

    model.eval()

    all_output = []

    for batch in dataloader:

        src = batch['src'].transpose(0, 1)
        trg = batch['tgt'].transpose(0, 1)

        batch_size = trg.shape[1]

        output, hidden = model.encoder(src)

        trg_indexes = [word2index['BOS']] * batch_size

        trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(0).to(device)

        outputs = torch.zeros(max_len, batch_size, dtype=torch.int)

        input = trg_tensor[0, :]

        for i in range(max_len):
            output, hidden = model.decoder(input, hidden)

            pred_token = output.argmax(1)
            outputs[i] = pred_token
            input = torch.LongTensor(pred_token).to(device)

        all_output.append(outputs.transpose(0, 1).numpy())

    return np.concatenate(all_output, axis=0)

In [38]:
all_output = generate(model, eval_loader, max_len=10)

In [39]:
def compute_rouge(decoded_preds, decoded_labels):
    rouge = Rouge()
    scores = []
    for i, j in zip(decoded_preds, decoded_labels):
        score = rouge.get_scores(' '.join(i), ' '.join(j))
        scores.append([
            score[0]['rouge-1']['f'], score[0]['rouge-2']['f'],
            score[0]['rouge-l']['f']
        ])
    rouge1 = np.mean([i[0] for i in scores])
    rouge2 = np.mean([i[1] for i in scores])
    rougel = np.mean([i[2] for i in scores])
    result = {
        "rouge1": round(rouge1, 4),
        "rouge2": round(rouge2, 4),
        "rouge-l": round(rougel, 4)
    }
    return result

In [40]:
preds = []
ytrues = []
for p, y in zip(all_output, test_targets):
    ps = []
    for j in p:
        if j in [word2index['BOS'], word2index['EOS'], word2index['PAD']]:
            continue
        ps.append(index2word[j])
    preds.append(ps)
    ys = []
    for j in y:
        if j in [word2index['BOS'], word2index['EOS'], word2index['PAD']]:
            continue
        ys.append(index2word[j])
    ytrues.append(ys)

In [41]:
compute_rouge(preds, ytrues)

{'rouge1': 0.0268, 'rouge2': 0.0, 'rouge-l': 0.0268}